## Autogen Studio


Autogen Studio is a web-based tool that lets you create AI agents visually -  no coding required. It's built on the Microsoft's Autogen framework, which is all about making smart agents that can chat, solve problems or even help with tasks.

### Installation

1. Create an environment (conda, venv or uv).
2. Activate the environment.
3. Create a .env file
4. Export your AZURE_OPENAI_API environment details.
5. run `autogenstudio ui` to start autogenstudio
6. To check the available commands use `autogenstudio ui --help`
7. In Autogen Studio settings add the following json to use Azure OpenAI Models:
    ```json
    {
        "provider": "autogen_ext.models.openai.AzureOpenAIChatCompletionClient",
        "component_type": "model",
        "version": 1,
        "component_version": 1,
        "description": "Chat completion client for Azure OpenAI hosted models.",
        "label": "Azure OpenAI gpt-4.1",
        "config": {
            "model": "gpt-4.1",
            "azure_deployment": "gpt-4.1",
            "azure_endpoint": "https://cxe-automations-hub.cognitiveservices.azure.com/",
            "api_key": "<api-key>",
            "api_version": "2024-12-01-preview"
        }
    }
    ```

### Autogen Components

1. Agent:
    - Name ---> A designated name to the agent.
    - Description ---> A suitable description that describes the task the agent is supposed to perform.
    - Model ---> The model that the agent would use to perform the task.
    - System Prompt ---> The set of instructions for the agent to accomplish the designated tasks successfully.
2. Tools
3. Model
4. Team: A Team can contain a single agent or multiple agents working together. So, for a team to work, it needs agent(s) and a Termination Condition.
5. Termination Condition

#### Some of the usual errors
- Error: instantiation Failed to instantiate component: Tools cannot be used with a workbench. Suggestion: Check that the component can be properly instantiated with the given config
- Fix: Delete the workbench property from the Assistant Agent.

### Create a custom Weather Tool using the OpenWeatherAPI
1. Add an environment variable called **OPENWEATHER_API_KEY** in Autogen Studio.
2. Create a new Tool in Autogen Studio Gallery.
3. Add the global imports necessary for your function.
4. Add the following python method.
    ```python
    def get_weather(location: str) -> str: 
            weather_api_key = os.getenv("OPENWEATHER_API_KEY")     
            url = f"http://api.openweathermap.org/data/2.5/weather?q={location}&appid={weather_api_key}&units=metric"
            

            response = requests.get(url)
            
            if response.status_code == 200:
                data = response.json()
                weather_description = data["weather"][0]["description"]
                temperature = data["main"]["temp"]
                humidity = data["main"]["humidity"]
                longitude = data["coord"]["lon"]
                latitude = data["coord"]["lat"]

                # Fetch air pollution data
                air_pollution_url = (
                    f"http://api.openweathermap.org/data/2.5/air_pollution?"
                    f"lat={latitude}&lon={longitude}&appid={weather_api_key}"
                )
                air_pollution_response = requests.get(air_pollution_url)

                if air_pollution_response.status_code == 200:
                    air_pollution_data = air_pollution_response.json()
                    air_quality_index = air_pollution_data["list"][0]["main"]["aqi"]

                    # Determine AQI status
                    aqi_status = {
                        1: "good",
                        2: "fair",
                        3: "moderate",
                        4: "poor",
                        5: "very poor",
                    }.get(air_quality_index, "unknown")

                    return (
                        f"The weather in {location} is {weather_description} with a temperature of {temperature}°C "
                        f"and a humidity of {humidity}%.\n"
                        f"The air quality index (AQI) is {air_quality_index}, which is considered {aqi_status}."
                    )
                else:
                    return f"Could not retrieve air pollution data for '{location}'."
            else:
                return f"Could not retrieve weather information for '{location}'."

    ```
5. Since we have added a new Environment variable to the Autogen Studio, make sure to restart the studio.